# Azure embeddings example
In this example we'll try to go over all operations for embeddings that can be done using the Azure endpoints. \
This example focuses on embeddings but also touches on the some other operations that are also available using the API. This example is meant to be a quick way of showing simple operations and is not meant as a tutorial.

In [ ]:
import openai
from openai import cli

## Setup
In the following section the endpoint and key need to be set up of the next sections to work. \
Please go to https://portal.azure.com, find your resource and then under "Resource Management" -> "Keys and Endpoints" look for the "Endpoint" value and one of the Keys. They will act as api_base and api_key in the code below.

In [ ]:
openai.api_key = ''  # Please add your api key here
openai.api_base = '' # Please add your endpoint here

openai.api_type = 'azure'
openai.api_version = '2022-12-01' # this may change in the future

## Deployments
In this section we are going to create a deployment that we can use to create embeddings.

### Deployments: Create Manually
Let's create a deployment using the `text-similarity-curie-001` engine. Create a new deployment by going to your Resource in your portal under "Resource Management" -> "Deployments".

### (Optional) Deployments: Create Programatically
We can also create a deployment using code:

In [ ]:
model = "text-similarity-curie-001"

# Now let's create the deployment
print(f'Creating a new deployment with model: {model}')
result = openai.Deployment.create(model=model, scale_settings={"scale_type":"standard"})
deployment_id = result["id"]

### (Optional) Deployments: Retrieving
Now let's check the status of the newly created deployment

In [ ]:
print(f'Checking for deployment status.')
resp = openai.Deployment.retrieve(id=deployment_id)
status = resp["status"]
print(f'Deployment {deployment_id} is with status: {status}')

### Deployments: Listing
Now because creating a new deployment takes a long time, let's look in the subscription for an already finished deployment that succeeded.

In [ ]:
print('While deployment running, selecting a completed one that supports embeddings.')
deployment_id = None
result = openai.Deployment.list()
for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])
    if model["capabilities"]["embeddings"] != True:
        continue
    
    deployment_id = deployment["id"]
    break

if not deployment_id:
    print('No deployment with status: succeeded found.')
else:
    print(f'Found a succeeded deployment that supports embeddings with id: {deployment_id}.')

### Embeddings
Now let's send a sample embedding to the deployment.

In [ ]:
embeddings = openai.Embedding.create(deployment_id=deployment_id,
                                     input="The food was delicious and the waiter...")
                                
print(embeddings)

### (Optional) Deployments: Delete
Finally let's delete the deployment

In [ ]:
print(f'Deleting deployment: {deployment_id}')
openai.Deployment.delete(sid=deployment_id)